<a href="https://colab.research.google.com/github/ShauryaK0303/TASK-1-SPAM-CLASSIFICATION/blob/main/LAB_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**AIM**
Implement the SVD++ algorithm and analyze it. Compare its
results with SVD algorithm.

#**INTRODUCTION**
SVD++ is an extension of Singular Value Decomposition (SVD) that incorporates implicitfeedback in addition to explicit feedback for matrix factorization. This extension improves theability of a recommendation system to predict ratings by utilizing not just the user-item ratings(explicit feedback), but also other signals such as clicks, views, purchases, or other userinteractions (implicit feedback).

#**THEORY**
SVD++ is an enhanced version of the Singular Value Decomposition (SVD) algorithm designed to improve recommendation accuracy by incorporating implicit feedback. While SVD uses explicit user-item ratings to factorize the user-item matrix, SVD++ adds implicit feedback, such as items a user interacted with but didn’t rate.

The model extends the user representation by including a term for implicit interactions, capturing hidden user preferences. This makes SVD++ more robust in scenarios where explicit ratings are sparse. The overall prediction formula includes both explicit and implicit feedback, allowing for more personalized and accurate recommendations.

In [2]:
#@title **CODE**
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357292 sha256=a615568f2386e1815e69c85b57640579cc8098630f4dc2f4c47d385ca0937e40
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import pandas as pd
from surprise import Dataset, Reader, SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import PredictionImpossible

In [5]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

In [6]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [7]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [8]:
model = SVDpp(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
model.fit(trainset)

In [9]:
predictions = model.test(testset)

In [10]:
rmse = accuracy.rmse(predictions)
print(f"Test RMSE: {rmse}")

RMSE: 0.9070
Test RMSE: 0.9069576349566727


In [15]:
def get_recommendations(user_id, top_n=10):
    all_movie_ids = ratings['movieId'].unique()
    rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId'].tolist()
    unrated_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids]
    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movie_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top_n recommended movie IDs
    recommended_movie_ids = [prediction.iid for prediction in predictions[:top_n]]

    return recommended_movie_ids

In [16]:
user_id = 10
recommendations = get_recommendations(user_id)
print(f"Recommendations for user {user_id}: {recommendations}")

Recommendations for user 10: [6711, 3275, 1208, 608, 1250, 8950, 103141, 913, 318, 904]


#**OUTPUT**
* SVD relies solely on explicit ratings, limiting its ability to fully capture
user preferences.
* SVD++ includes implicit feedback, leading to more accurate predictions,especially for users or items with sparse explicit ratings.

* SVD++ slightly adjusts the ratings based on implicit interactions, improving recommendation relevance.
* For example, U1's prediction for Item 5 improves with SVD++ because of implicit feedback from similar users/items.

#**ANALYSIS & CONCLUSION**
SVD++ improves upon the traditional SVD algorithm by incorporating implicit feedback such as clicks, views, or purchases alongside explicit ratings. This additional data enables the algorithm to capture nuanced user preferences, especially when explicit ratings are sparse. By considering implicit interactions, SVD++ generates more personalized recommendations and performs better in predicting missing ratings.

SVD++ is a powerful enhancement of SVD, offering improved accuracy by leveraging both explicit and implicit feedback. It is particularly effective in real-world scenarios where user behavior data is abundant but explicit ratings are limited. While computationally intensive, its ability to provide superior recommendations makes it a valuable choice for modern recommendation systems.